**Настройка макета записаной книжки -> Output: Word Wrap -> Вкл.**

### Что изменено?
- ~~ХЗ, пока вроде ничего, просто код почище сделал~~
- В качестве классификатора используется голосующий ансамбль из модели логистической регрессии, модели леса случайных деревьев и модели градиентного бустинга 
- "Поиграйте ~~со шрифтами~~ с параметрами!"

In [1]:
# РОТ ШАТАЛ ЭТИХ ЗАВИСИМОСТЕЙ ПИТОНА

# !sudo apt remove python* -y
# !sudo apt install python3 pip -y
# !/bin/python3 -m pip install ipykernel -U --user --force-reinstall
# !pip freeze | cut -d "@" -f1 | xargs pip uninstall -y

# !pip uninstall datasets spacy scikit-learn -y
# !pip uninstall thinc numpy scipy -y
# !pip install "thinc>8.3" "numpy<2.0" scipy
# !pip install scikit-learn spacy datasets

In [2]:
!python3 -m spacy download ru_core_news_sm

In [3]:
import numpy as np
from typing import List, Tuple

import sklearn.metrics as m
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, StackingClassifier

import spacy
from datasets import load_dataset
import multiprocessing

### 1. Загрузим и разделим текстовый корпус

In [4]:
def load_sib200_ru() -> Tuple[Tuple[List[str], List[int]], Tuple[List[str], List[int]], Tuple[List[str], List[int]], List[str]]:
    trainset = load_dataset('Davlan/sib200', 'rus_Cyrl', split='train')
    X_train = trainset['text']
    y_train = trainset['category']

    valset = load_dataset('Davlan/sib200', 'rus_Cyrl', split='validation')
    X_val = valset['text']
    y_val = valset['category']

    testset = load_dataset('Davlan/sib200', 'rus_Cyrl', split='test')
    X_test = testset['text']
    y_test = testset['category']


    categories = set(y_train)

    unknown_categories = set(y_val) - set(y_train)
    if len(unknown_categories) > 0:
        raise RuntimeError(
            f'Категории {unknown_categories} есть в валидационном наборе, но не в тренировочном!'
        )

    unknown_categories = set(y_test) - set(y_train)
    if len(unknown_categories) > 0:
        raise RuntimeError(
            f'Категории {unknown_categories} есть в тестовом наборе, но не в тренировочном!'
        )


    categories = sorted(list(categories))

    y_train = [categories.index(it) for it in y_train]
    y_val = [categories.index(it) for it in y_val]
    y_test = [categories.index(it) for it in y_test]

    return (X_train, y_train), (X_val, y_val), (X_test, y_test), categories

In [5]:
train_data, val_data, test_data, classes_list = load_sib200_ru()

In [6]:
print('Категории текстового корпуса:', classes_list)

Категории текстового корпуса: ['entertainment', 'geography', 'health', 'politics', 'science/technology', 'sports', 'travel']


In [7]:
print(len(train_data[0]), len(train_data[1]))
print(len(val_data[0]), len(val_data[1]))
print(len(test_data[0]), len(test_data[1]))

701 701
99 99
204 204


### 2. Применим SpaCy для предобработки текста

In [8]:
def normalize_text(s: str, nlp_pipeline: spacy.Language) -> str:
    doc = nlp_pipeline(s)
    lemmas = [('<NUM>' if token.like_num else token.lemma_.lower()) for token in filter(lambda it1: not it1.is_punct, doc)]
    if len(lemmas) == 0:
        return ''
    return ' '.join(lemmas)

In [9]:
NLP_PIPELINE = spacy.load('ru_core_news_sm')

In [10]:
print(
    train_data[0][0],
    '\n' + normalize_text(train_data[0][0], NLP_PIPELINE),
)
print(
    '\n' + val_data[0][0],
    '\n' + normalize_text(val_data[0][0], NLP_PIPELINE),
)
print(
    '\n' + test_data[0][0],
    '\n' + normalize_text(test_data[0][0], NLP_PIPELINE),
)

Турция с трёх сторон окружена морями: на западе — Эгейским, на севере — Чёрным и на юге — Средиземным. 
турция с <NUM> сторона окружить морями на запад эгейским на север чёрный и на юг средиземный

Если увеличить расстояние для бега с четверти до половины мили, скорость становится не так важна, тогда как выносливость превращается в абсолютную необходимость. 
если увеличить расстояние для бег с <NUM> до <NUM> миля скорость становиться не так важный тогда как выносливость превращаться в абсолютный необходимость

Мутация вносит новую генетическую вариацию, в то время как отбор убирает её из набора проявляющихся вариаций. 
мутация вносить новый генетический вариация в тот время как отбор убирать её из набор проявляться вариация


In [11]:
class_probability = 1.0 / len(classes_list)
max_df = 1.0 - 0.2 * class_probability

print(f'Максимально возможная документная частота: {max_df}')

Максимально возможная документная частота: 0.9714285714285714


### 3. Применим какой-нибудь классификатор

In [128]:
classifier = Pipeline(steps=[
    # 1. Перевод текста в векторное пространство
    ('vectorizer', TfidfVectorizer(
        token_pattern=r'\b\w+\b',
        sublinear_tf=True,
    )),
    # ('cls', LogisticRegression(solver='saga', max_iter=1000, random_state=42))
    # 2. Регрессионное наложение многих классификаторов
    ('classifier', VotingClassifier(
        estimators=[
            ('lr', LogisticRegression(
                max_iter=100,
                random_state=42,
                class_weight='balanced',
                penalty='l2',
                solver='liblinear',
            )),
            ('rf', RandomForestClassifier(
                n_estimators=100,
                random_state=42,
                class_weight='balanced',
            )),
            # ('cb', CatBoostClassifier(
            #     # task_type="GPU", devices='0', # CUDA out of memory (why???)
            #     iterations=100,
            #     random_state=42,
            #     depth=10,
            #     verbose=0,
            # ))
            ('xgb', XGBClassifier(
                n_estimators=100,
                random_state=42,
            ))
        ],
        voting='soft',
        n_jobs=-1, # Юзаем все ядра CPU
        verbose=True
    ))
])

In [129]:
CV = GridSearchCV( # Перебор параметров для нахождения дающих лучший (согласно оценке) результат
    estimator=classifier,

    param_grid={
        # Подбираем параметры TFIDF
        # 'vectorizer__ngram_range': [(1, 1), (1, 2), (1, 3)], # --> (1, 1)
        # 'vectorizer__max_features': [4000, 5000, 6000], # Не делаем ограничения
        'vectorizer__max_df': [0.4, 0.6, 0.8],
        'vectorizer__min_df': [0.5, 1, 1.5],

        # Подбираем параметры лог. рег-сии
        'classifier__lr__C': [1000, 10000, 100000],
        # 'classifier__lr__solver': ['saga', 'liblinear'], # --> liblinear
        # 'classifier__lr__penalty': ['l1', 'l2'], # --> l2

        # Подбираем параметры случ. леса
        'classifier__rf__max_depth': [5, 10, 15],

        # Подбираем параметры catboost-а
        # 'classifier__cb__learning_rate': [0.1, 0.2, 0.3],
        # 'classifier__cb__depth': [6, 8, 10],

        # # Подбираем параметры xgboost-а
        'classifier__xgb__learning_rate': [0.05, 0.10, 0.15],
        'classifier__xgb__max_depth': [5, 10, 15],
    },

    scoring='f1_macro',
    cv=5,
    refit=True,
    n_jobs=-1, # Юзаем все ядра CPU
    verbose=True
)

In [130]:
CV.fit([normalize_text(it, NLP_PIPELINE) for it in train_data[0]], train_data[1])

Fitting 5 folds for each of 729 candidates, totalling 3645 fits
[Voting] ....................... (1 of 3) Processing lr, total=   0.0s
[Voting] ....................... (1 of 3) Processing lr, total=   0.1s
[Voting] ....................... (1 of 3) Processing lr, total=   0.1s
[Voting] ....................... (1 of 3) Processing lr, total=   0.0s[Voting] ....................... (1 of 3) Processing lr, total=   0.1s

[Voting] ....................... (1 of 3) Processing lr, total=   0.0s
[Voting] ....................... (1 of 3) Processing lr, total=   0.0s
[Voting] ....................... (2 of 3) Processing rf, total=   0.2s
[Voting] ....................... (1 of 3) Processing lr, total=   0.0s
[Voting] ...................... (3 of 3) Processing xgb, total=   0.1s
[Voting] ....................... (2 of 3) Processing rf, total=   0.3s
[Voting] ...................... (3 of 3) Processing xgb, total=   0.2s
[Voting] ....................... (2 of 3) Processing rf, total=   0.2s
[Voting] ....

[Voting] ....................... (1 of 3) Processing lr, total=   0.1s
[Voting] ....................... (2 of 3) Processing rf, total=   0.5s
[Voting] ...................... (3 of 3) Processing xgb, total=  12.0s
[Voting] ....................... (1 of 3) Processing lr, total=   0.0s
[Voting] ....................... (2 of 3) Processing rf, total=   0.3s
[Voting] ...................... (3 of 3) Processing xgb, total=  10.6s
[Voting] ....................... (1 of 3) Processing lr, total=   0.1s
[Voting] ...................... (3 of 3) Processing xgb, total=   3.1s
[Voting] ....................... (1 of 3) Processing lr, total=   0.0s[Voting] ....................... (2 of 3) Processing rf, total=   0.4s

[Voting] ....................... (2 of 3) Processing rf, total=   0.3s
[Voting] ...................... (3 of 3) Processing xgb, total=   2.8s
[Voting] ....................... (1 of 3) Processing lr, total=   0.1s
[Voting] ...................... (3 of 3) Processing xgb, total=  10.6s
[Votin

1620 fits failed out of a total of 3645.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
405 fits failed with the following error:
ValueError: max_df corresponds to < documents than min_df

--------------------------------------------------------------------------------
1215 fits failed with the following error:
 0.09173114 0.63207837        nan        nan 0.62877333        nan
 0.09173114 0.63940244        nan 0.09173114 0.63940244        nan
        nan 0.63183424        nan 0.09173114 0.63971928        nan
 0.09173114 0.63971928        nan        nan 0.63114659        nan
 0.09173114 0.6324108         nan 0.09173114 0.6324108         nan
        nan 0.62523738        nan 0.09173114 0.62817934        nan
 0.09173114 0.62817934        n

[Voting] ....................... (2 of 3) Processing rf, total=   0.1s
[Voting] ...................... (3 of 3) Processing xgb, total=   2.1s


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('vectorizer',
                                        TfidfVectorizer(sublinear_tf=True,
                                                        token_pattern='\\b\\w+\\b')),
                                       ('classifier',
                                        VotingClassifier(estimators=[('lr',
                                                                      LogisticRegression(class_weight='balanced',
                                                                                         random_state=42,
                                                                                         solver='liblinear')),
                                                                     ('rf',
                                                                      RandomForestClassifier(class_weight='balanced',
                                                                                             random_state=42)),
                                                                     ('xgb',
                                                                      XGBClassifier(base_sco...
                                                                                    random_state=42, ...))],
                                                         n_jobs=-1,
                                                         verbose=True,
                                                         voting='soft'))]),
             n_jobs=-1,
             param_grid={'classifier__lr__C': [1000, 10000, 100000],
                         'classifier__rf__max_depth': [5, 10, 15],
                         'classifier__xgb__learning_rate': [0.05, 0.1, 0.15],
                         'classifier__xgb__max_depth': [5, 10, 15],
                         'vectorizer__max_df': [0.4, 0.6, 0.8],
                         'vectorizer__min_df': [0.5, 1, 1.5]},
             scoring='f1_macro', verbose=True)

In [131]:
print('Лучшие параметры:', CV.best_params_)
print('Лучший показатель F1:', CV.best_score_)
print('Размер словаря в векторном пр-ве:', len(CV.best_estimator_.named_steps["vectorizer"].vocabulary_))

Лучшие параметры: {'classifier__lr__C': 100000, 'classifier__rf__max_depth': 5, 'classifier__xgb__learning_rate': 0.05, 'classifier__xgb__max_depth': 5, 'vectorizer__max_df': 0.6, 'vectorizer__min_df': 1}
Лучший показатель F1: 0.6523776699807552
Размер словаря в векторном пр-ве: 4359


In [132]:
y_pred_val = CV.predict([normalize_text(it, NLP_PIPELINE) for it in val_data[0]])

print(m.classification_report(
    y_true=val_data[1],
    y_pred=y_pred_val,
    target_names=classes_list
))

                    precision    recall  f1-score   support

     entertainment       0.80      0.44      0.57         9
         geography       0.60      0.75      0.67         8
            health       1.00      0.55      0.71        11
          politics       0.90      0.64      0.75        14
science/technology       0.61      0.80      0.69        25
            sports       0.75      0.75      0.75        12
            travel       0.57      0.65      0.60        20

          accuracy                           0.68        99
         macro avg       0.75      0.65      0.68        99
      weighted avg       0.72      0.68      0.68        99



In [133]:
y_pred_test = CV.predict([normalize_text(it, NLP_PIPELINE) for it in test_data[0]])

print(m.classification_report(
    y_true=test_data[1],
    y_pred=y_pred_test,
    target_names=classes_list
))

                    precision    recall  f1-score   support

     entertainment       0.88      0.37      0.52        19
         geography       0.77      0.59      0.67        17
            health       0.43      0.45      0.44        22
          politics       0.80      0.80      0.80        30
science/technology       0.66      0.80      0.73        51
            sports       0.84      0.84      0.84        25
            travel       0.60      0.65      0.63        40

          accuracy                           0.68       204
         macro avg       0.71      0.64      0.66       204
      weighted avg       0.70      0.68      0.68       204

